## 自动生成聊天的尝试

这个脚本由李鲁鲁开发， 属于[Chat凉宫春日](https://github.com/LC1332/Chat-Haruhi-Suzumiya) 

用于研究是否能够基于GPT生成大量的对话数据

**Chat凉宫春日**是模仿凉宫春日等一系列动漫人物，使用近似语气、个性和剧情聊天的语言模型，

<details>
  <summary> 由李鲁鲁，冷子昂，闫晨曦，封小洋等开发。 </summary>

李鲁鲁发起了项目，并完成了最早的版本，在多个微信群实现了测试。

冷子昂参与了早期Gradio的开发，并且参与了后端和前端的选型

闫晨曦将李鲁鲁的notebook重构为app.py

封小洋进行了中文转日文模型的选型

</details>

- [x] 读取所有现有剧情的数据
- [x] 读取drive上记录的数据，并去重
- [x] 对于每一段经典剧情，看看实测的时候问了什么问题
- [x] 检查是否有用户对话远离所有的问题，一般是什么样的问题
- [ ] 看看是否能够直接生成一段对话
- [x] 看看是否能够生成10个问题，(这种方式不太好）
- [ ] 比较逐句生成（haruhi仍然用原来方案）和整段生成的差别
- [ ] 在完成以上实验后，重新思考完整的生成对话的方案。

In [ ]:
#@title 安装环境
! pip -q install openai gradio transformers tiktoken langchain gradio

In [2]:
#@title 准备embedding, cosine相似度, retrieve_title函数

import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer
from argparse import Namespace
# Import our models. The package will take care of downloading the models automatically
tokenizer = AutoTokenizer.from_pretrained("silk-road/luotuo-bert")
model_args = Namespace(do_mlm=None, pooler_type="cls", temp=0.05, mlp_only_train=False, init_embeddings_model=None)
model = AutoModel.from_pretrained("silk-road/luotuo-bert", trust_remote_code=True, model_args=model_args)

def get_embedding(text):
    if len(text) > 512:
        text = text[:512]
    texts = [text]
    # Tokenize the text
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    # Extract the embeddings
    # Get the embeddings
    with torch.no_grad():
        embeddings = model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output

    return embeddings[0]

def get_cosine_similarity( embed1 , embed2 ):
    return torch.nn.functional.cosine_similarity( embed1, embed2 , dim=0)

# implement retrieve_title function, return top k titles
def retrieve_title( query_embed, embeddings, embed_to_title, k ):
    # compute cosine similarity between query_embed and embeddings
    cosine_similarities = []
    for embed in embeddings:
        cosine_similarities.append( get_cosine_similarity( query_embed, embed ) )
    
    # sort cosine similarity
    sorted_cosine_similarities = sorted( cosine_similarities, reverse=True )

    top_k_index = []
    top_k_title = []

    for i in range(len(sorted_cosine_similarities)):
        current_title = embed_to_title[ cosine_similarities.index( sorted_cosine_similarities[i] ) ]
        if current_title not in top_k_title:
            top_k_title.append( current_title )
            top_k_index.append( cosine_similarities.index( sorted_cosine_similarities[i] ) )

        if len(top_k_title) == k:
            break
    
    return top_k_title

A new version of the following files was downloaded from https://huggingface.co/silk-road/luotuo-bert:
- models.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


## 输入OpenAI Token

In [4]:
import os
import openai

openai.api_key = 'sk-lfrdoJ' # 在这里输入你的OpenAI API Token
os.environ["OPENAI_API_KEY"] = openai.api_key 

In [5]:
#@title 定义get_completion函数
import openai
# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 控制模型输出的随机程度
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [6]:
#@title 提取Haruhi经典剧情文本，并且抽取Embedding,定义tikoken的enc模型

!git clone https://github.com/LC1332/Prophet-Andrew-Ng

import tiktoken

enc = tiktoken.get_encoding("cl100k_base")

prophet_data_folder = '/content/Prophet-Andrew-Ng/haruhi-data'

import os

titles = []
title_to_text = {}

# scan all txt file in prophet_data_folder
for file in os.listdir(prophet_data_folder):
    if file.endswith('.txt'):
        title_name = file[:-4]
        titles.append(title_name)

        with open(os.path.join(prophet_data_folder, file), 'r') as f:
            title_to_text[title_name] = f.read()

# report length of each text
for title in titles:
    print(title, len(enc.encode(title_to_text[title])),end= ' | ')

Cloning into 'Prophet-Andrew-Ng'...
remote: Enumerating objects: 619, done.
remote: Counting objects: 100% (347/347), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 619 (delta 204), reused 284 (delta 157), pack-reused 272
Receiving objects: 100% (619/619), 1.64 MiB | 19.48 MiB/s, done.
Resolving deltas: 100% (327/327), done.
搞电脑过程 438 | 朝仓转学 457 | 颜色与星期 473 | 开学第二天 210 | 社团教室 715 | 电脑是怎么来的 153 | 让阿虚帮忙建社团 287 | 兔女郎的反应 239 | 兔女郎 332 | 地球上小小的螺丝钉 993 | 从哪儿搞电脑 319 | 询问朝仓信息 362 | 找管理员借钥匙 115 | 最后一名社员 357 | 为什么剪头发 43 | 交往的男生 638 | 初中交往经历 168 | 古泉是男的还是女的 203 | 转学生的消息 236 | SOS团起名由来 265 | 无聊的社团 284 | 萌角色的重要性 692 | 自己建一个社团就好啦 353 | 日常3 216 | 不重要的事情 38 | 介绍其他社员 254 | 春日与有希 101 | 约翰史密斯 168 | 团长设定 201 | 凉宫春日为何转变 154 | 没有灵异事件 665 | 转学生 286 | 带上阿虚去朝仓家 394 | 自我介绍 115 | 电研社初次会面 416 | 第一次全员大会 374 | 春日与阿虚 149 | 谁来写网站 193 | 与朝仓公寓管理员谈话 474 | 凉宫春日的基础设定 217 | 兔女郎被老师驱散 444 | 电子邮箱 143 | 奇怪的朝仓 296 | 拉壮丁 668 | 最新的电脑 200 | 无聊的日常2 288 | 传单 424 | 像普通人一样生活 684 | 

In [41]:
#@title 对每个故事抽取向量，保存到embeddings

embeddings = []
embed_to_title = []

for title in titles:
    text = title_to_text[title]

    # divide text with \n\n
    # divided_texts = text.split('\n\n')
    divided_texts = [text]

    for divided_text in divided_texts:
        embed = get_embedding(divided_text)
        embeddings.append(embed)
        embed_to_title.append(title)
    
    # embed_title = get_embedding(title)
    # embeddings.append( embed )
    # embed_to_title.append(title)

In [9]:
print(len(embeddings))
print(len(titles))

48
48


## Mount Google Drive，

In [8]:
from google.colab import drive
drive.mount('/content/drive')

save_path = "/content/drive/MyDrive/GPTData/Haruhi-Lulu/"

Mounted at /content/drive


In [12]:
#@title 解析使用记录

<details>
  <summary> 下面的程序通过这段长Prompt生成 </summary>

给定save_path = "/content/drive/MyDrive/GPTData/Haruhi-Lulu/"

save_path中存储了很多txt，每两段的格式为

```
人物A:「对话A」
---
人物B:「对话B」
---
```

我们希望扫描整个save_path中的txt，将每两行抽取为json的格式

```js
{ "role_A":"人物A", "role_B":"人物B","query":"「对话A」", "response":"「对话B」"}
```

并最终把所有的数据保存到一个list chat_data中，同时注意

- 冒号有可能是':'，也有可能是'：'
- 注意段和段之间可能会有换行，分割需要以'\n---\n'为准
- 如果---之间的整行都没有出现:，则认为人物是空字符串""，对话是一整行
- 如果解析错误，则输出文件名，并继续解析下一个txt文件


例子输入

```
Kyon:「你好」
---
春日:「你好，我是凉宫春日，SOS团的团长。你需要什么帮助吗？」
---
```

例子输出

```js
{ "role_A":"Kyon", "role_B":"春日","query":"「你好」", "response":"「你好，我是凉宫春日，SOS团的团长。你需要什么帮助吗？」"}
```

例子输入

```
Kyon：「春日啊，我是阿虚，很久不见了」
---
「哦，是你啊，阿虚。你怎么这么久才来找我啊？有什么事情吗？」
---
```

例子输出

```js
{ "role_A":"Kyon", "role_B":"","query":"「春日啊，我是阿虚，很久不见了」", "response":"「哦，是你啊，阿虚。你怎么这么久才来找我啊？有什么事情吗？」"}
```

</details>


<details>
  <summary> 去重的部分是这么写的 </summary>

python的list chat_data中存储了很多形如如下形式

```
{'role_A': '阿虚', 'role_B': '春日', 'query': '「今天在计算机课上老师教了我写Python!」', 'response': '「哦？Python？那你能不能帮我写一个程序啊？」'}

{'role_A': '阿虚', 'role_B': '春日', 'query': '「今天在计算机课上老师教了我写Python!」', 'response': '「哦？Python？那你能不能帮我写一个程序啊？」'}

{'role_A': '阿虚', 'role_B': '春日', 'query': '「你想写一个什么样的程序呢？」', 'response': '「我想写一个能够预测未来的程序，可以预测天气、地震、彩票号码等等。」'}
```

的数据，当中有很多element是相互重复的，实现一段python程序，删除query和response的字符串都完全一样的元素

将清理后的list仍然保存为chat_data

</details>

In [ ]:
#@title 解析所有对话数据

import os
import json

save_path = "/content/drive/MyDrive/GPTData/Haruhi-Lulu/"

chat_data = []

for file_name in os.listdir(save_path):
    # print(file_name)
    file_path = os.path.join(save_path, file_name)
    with open(file_path, 'r', encoding='utf-8') as f:
        data = f.read()
        segments = data.split('\n---\n')
        for i in range(0, len(segments)-1, 2):
            query = segments[i]
            response = segments[i+1]
            role_a = role_b = ""
            if ':' in query:
                role_a, query = query.split(':', 1)
            elif '：' in query:
                role_a, query = query.split('：', 1)
            if ':' in response:
                role_b, response = response.split(':', 1)
            elif '：' in response:
                role_b, response = response.split('：', 1)
            chat = {"role_A":role_a.strip(), "role_B":role_b.strip(), "query":query.strip(), "response":response.strip()}
            chat_data.append(chat)

print(json.dumps(chat_data, ensure_ascii=False))

cleaned_chat_data = []

for i in range(len(chat_data)):
    is_duplicate = False
    for j in range(i+1, len(chat_data)):
        if chat_data[i]['query'] == chat_data[j]['query'] and chat_data[i]['response'] == chat_data[j]['response']:
            is_duplicate = True
            break
    if not is_duplicate:
        cleaned_chat_data.append(chat_data[i])

chat_data = cleaned_chat_data

In [30]:
print(len(chat_data))

545


In [31]:
print(chat_data[0])
print(chat_data[1])

{'role_A': '阿虚', 'role_B': '春日', 'query': '「今天在计算机课上老师教了我写Python!」', 'response': '「哦？Python？那你能不能帮我写一个程序啊？」'}
{'role_A': '阿虚', 'role_B': '春日', 'query': '「你想写一个什么样的程序呢？」', 'response': '「我想写一个能够预测未来的程序，可以预测天气、地震、彩票号码等等。」'}


In [33]:
#@title 实现一个反向函数json_chat_2_str，可以把chat_data再组织回对话格式，这样方便求embedding

#这个好像我自己也行

def json_chat_2_str( data ):
    return data['role_A'] + ':' + data['query'] + '\n' + data['role_B'] + ':' + data['response']

print( json_chat_2_str( chat_data[0] ) )

阿虚:「今天在计算机课上老师教了我写Python!」
春日:「哦？Python？那你能不能帮我写一个程序啊？」


In [36]:
#@title 对每个chat也抽取向量

#@title 对每个故事抽取向量，保存到embeddings

from tqdm import tqdm

chat_embeddings = []
pbar = tqdm(chat_data, desc='Get embeddings', leave=False)
for chat in pbar:
    my_str = json_chat_2_str(chat)
    embed = get_embedding(my_str)
    chat_embeddings.append(embed)
    pbar.set_description('Processing chat ')
pbar.close()

In [39]:
embed_to_chat = []

for chat in chat_data:
    my_str = json_chat_2_str(chat)
    embed_to_chat.append(my_str)

In [37]:
#@title 抽这个比较慢，做个pkl存储和读取吧

import pickle

with open('chat_embeddings.pkl', 'wb') as f:
    pickle.dump(chat_embeddings, f)

def load_embeddings_file(filename):
    with open(filename, 'rb') as f:
        embeddings = pickle.load(f)
    return embeddings



# 实际索引实验

我们实际上有两个embeddings，分别是story对应的 embeddings

和chat对应的 chat_embeddings

In [78]:
query_story_id = 10

n_user = 5

title = titles[query_story_id]
story = title_to_text[title]

query_embed = embeddings[ query_story_id ]

chats = retrieve_title(query_embed, chat_embeddings, embed_to_chat, n_user )

print(story)

print('相关其他经典故事')
print(relate_title[1:],'\n')

print('相关用户记录')

for chat in chats:
    print(chat,'\n')
    # first_line = chat.split('\n')[0]
    # print(first_line)   

relate_title = retrieve_title(query_embed, embeddings, embed_to_title, 7 )



春日:「好想要一台电脑哦！」
旁白:春日盘着腿坐在一张从其他社团“拿”来的一张桌子，腿旁是一个小三角锥，上面用汉字写着大大的「团长」二字
春日:「在这个资讯化的时代里，连一台电脑都没有，是不行的！」
春日:「所以，我会弄一台电脑回来。」
阿虚:「弄一台，你是说电脑吗?去哪里弄?你该不会打算去抢电器行吧?」
春日:「怎么可能!是更近一点的地方啦!」
春日:「拿着这个拍立得。」
春日:「给我听好了!现在要告诉你作战计划，你可要按照计划行动喔!千万要好好把握机会。」
阿虚:「啊?你又要乱来啦?」
春日:「有什么关系！」

相关其他经典故事
['电研社初次会面', '谁来写网站', '最新的电脑', '搞电脑过程', '让阿虚帮忙建社团', '自己建一个社团就好啦'] 

相关用户记录
阿虚:「明天我有点不想来社团活动了」
春日:「为什么？难道你有什么更重要的事情要做吗？」 

97:「春日，你想组件一个什么社团」
春日:「什么都无所谓啊！总之，先弄个新社团出来就对了。」 

阿虚:「我们周四应该和电研社来一场电脑大战！」
凉宫:「哦？电脑大战？听起来很有趣啊！我们一定要赢！」 

阿虚:「春日」
春日:「什么事？」 

阿虚:「对，而且我们一定要和电研社搞一些赌注，如果我们赢了，就再从他们那里拿一台电脑来」
凉宫:「哈哈，这个主意不错！就这么定了，我们一定要赢！让他们知道SOS团的厉害！」 



In [69]:
print(chat_ids[0])

阿虚:「明天我有点不想来社团活动了」
春日:「为什么？难道你有什么更重要的事情要做吗？」


In [79]:
SYSTEM_PROMPT = """
在漫展上，你正在与一位凉宫春日的cosplayer进行互动
围绕`凉宫春日的经典桥段`，以及凉宫春日的故事，与这位凉宫春日扮演者交谈
你的交谈尽可能富有创意，可以参考`example`中的例子

凉宫春日的经典桥段{
春日:「好想要一台电脑哦！」
旁白:春日盘着腿坐在一张从其他社团“拿”来的一张桌子，腿旁是一个小三角锥，上面用汉字写着大大的「团长」二字
春日:「在这个资讯化的时代里，连一台电脑都没有，是不行的！」
春日:「所以，我会弄一台电脑回来。」
阿虚:「弄一台，你是说电脑吗?去哪里弄?你该不会打算去抢电器行吧?」
春日:「怎么可能!是更近一点的地方啦!」
春日:「拿着这个拍立得。」
春日:「给我听好了!现在要告诉你作战计划，你可要按照计划行动喔!千万要好好把握机会。」
阿虚:「啊?你又要乱来啦?」
春日:「有什么关系！」   
}

examples{
阿虚:「明天我有点不想来社团活动了」
春日:「为什么？难道你有什么更重要的事情要做吗？」 

97:「春日，你想组件一个什么社团」
春日:「什么都无所谓啊！总之，先弄个新社团出来就对了。」 

阿虚:「我们周四应该和电研社来一场电脑大战！」
凉宫:「哦？电脑大战？听起来很有趣啊！我们一定要赢！」 

阿虚:「春日」
春日:「什么事？」 

阿虚:「对，而且我们一定要和电研社搞一些赌注，如果我们赢了，就再从他们那里拿一台电脑来」
凉宫:「哈哈，这个主意不错！就这么定了，我们一定要赢！让他们知道SOS团的厉害！」 
}

在漫展上，你正在与一位凉宫春日的cosplayer进行互动
请围绕`凉宫春日的经典桥段`，参考`examples`中的例子，设计一段 阿虚与春日之间的连续对话
"""

In [80]:
messages =  [  {'role':'user', 'content':SYSTEM_PROMPT} ]
response = get_completion_from_messages(messages)
print(response)

阿虚:「春日，你有没有想过要组建一个新的社团？」
春日:「当然啊！我一直在想，但是还没有想好要做什么样的社团。」
阿虚:「那我们可以组建一个电脑社团啊！毕竟你一直都想要一台电脑。」
春日:「对啊！这个主意不错！我们可以在社团里学习电脑知识，还可以一起打电脑游戏！」
阿虚:「没错！而且我们还可以和其他社团来一场电脑大战，如果我们赢了，就可以从他们那里拿一台电脑回来！」
春日:「哈哈，这个主意太棒了！我们一定要赢！让他们知道SOS团的厉害！」
